Hi, markdown is cool. Anyway let's get started with some code.

Here is a link to a more in depth of the definitions: https://docs.google.com/document/d/17I6Z9MHgMIAgO7UkEsvGndTIKirIrb9lF6upHfAi0iw/edit?usp=sharing


In [26]:
class Token:
    def __init__(self, code:int, lexeme:str):
        self.code = code

In [27]:
class Lexer:
    def __init__(self,input:str):
        # input is the file in a single string
        self.input = input
        # token_map is a dictionary that maps the tokens to their codes
        self.tokenList = {
           1: Token(1, 'real_literal'),
           2:Token(2, 'integer_literal'),
           3: Token(3, 'bool_literal'),
           4: Token(4, 'char_literal'),
           5: Token(5, 'string_literal'), 
           6: Token(6, 'if'),
           7: Token(7, 'else'),
           8: Token(8, 'while'),
           9: Token(9, 'for'),
           10: Token(10, 'do'),
           11: Token(11, 'var'),
           12: Token(12, 'int'),
           13: Token(13, 'bool'),
           14: Token(14, 'char'),
           15: Token(15, 'string'),
           16: Token(16, 'real'),
           17: Token(17, '+'),
           18: Token(18, '-'),
           19: Token(19, '*'),
           20: Token(20, '/'),
           21: Token(21, '%'),
           22: Token(22, '='),
           23: Token(23, '=='),
           24: Token(24, '!=='),
           25: Token(25, '<'),
           26: Token(26, '>'),
           27: Token(27, '<=='),
           28: Token(28, '>=='),
           29: Token(29, '&&'),
           30: Token(30, '||'),
           31: Token(31, '!'),
           32: Token(32, '**'),
           33: Token(33, '('),
           34: Token(34, ')'),
           35: Token(35, '{'),
           36: Token(36, '}'),
           37: Token(37, '~'),
           38: Token(38, 'function'),
           39: Token(39, 'id')
        }
        self.token_map = {'real_literal':1,
                        'integer_literal':2,
                        'bool_literal':3,
                        'char_literal':4,
                        'string_literal':5,
                        'if':6,
                        'else':7,
                        'while':8,
                        'for':9,
                        'do':10,
                        'var':11,
                        'int':12,
                        'bool':13,
                        'char':14,
                        'string':15,
                        'real':16,
                        '+':17,
                        '-':18,
                        '*':19,
                        '/':20,
                        '%':21,
                        '=':22,
                        '==':23,
                        '!==':24,
                        '<':25,
                        '>':26,
                        '<==':27,
                        '>==':28,
                        '&&':29,
                        '||':30,
                        '!':31,
                        '**':32,
                        '(':33,
                        ')':34,
                        '{':35,
                        '}':36,
                        '~':37,
                        'function':38,
                        'id':39,
        }
    
    def find_tokens(self): 
        # finds all the tokens in a line
        tokens = []
        for i in self.input.split():
            if i in self.token_map:
                tokens.append(self.tokenList[self.token_map[i]])
            else:
                tokens.append(self.tokenList[self.token_map['id']])
        return tokens
    
            
    def error(self):
        # stops the program from running
        raise ValueError("Invalid token")

In [28]:
class Parser:
    """ Takes in a list of Token object in its constructor
    Outputs a parse tree of called functions that would recognize the input is syntactically correct """
    def __init__(self,tokens:list):
        self.tokens = tokens
        if self.empty():
            print("Empty file")
        else:
            self.program()
    def error(self):
        raise ValueError("Syntax Error")
    def program(self):
        #<program> --> <stmt>
        self.stmt() 
    def stmt(self):
        #<stmt> --> <block> | <if_stmt> | <assignment> |<empty>
        if self.tokens[0].code == 35:
            self.block()
        elif self.tokens[0].code == 6:
            self.if_stmt()
        elif self.tokens[0].code == 11:
            self.assignment()
        elif self.tokens[0].code == 38:
            self.functions()
        else:
            self.error()

    def empty(self):
        #<empty> --> ''
        if len(self.tokens) == 0:
            return True
    
    def block(self):
        #<block> --> `{`{<stmt>}`}`
        if self.tokens[0].code == 35:
            self.tokens.pop(0)
            while self.tokens[0].code != 36:
                self.stmt()
            self.tokens.pop(0)
        else:
            self.error()
    
    def loop(self):
        # <loop> --> <while_loop> | <do_while> | <for_loop>
        if self.tokens[0].code == 8:
            self.while_loop()
        elif self.tokens[0].code == 10:
            self.do_while()
        elif self.tokens[0].code == 9:
            self.for_loop()
        else:
            self.error()
    
    def if_stmt(self):
        # <if_stmt>   -->  `if``(`<bool_stmt> `)`<stmt>[`else ` <stmt>]
        if self.tokens[0].code == 6:
            self.tokens.pop(0)
            if self.tokens[0].code == 33:
                self.tokens.pop(0)
                self.bool_stmt()
                if self.tokens[0].code == 34:
                    self.tokens.pop(0)
                    self.stmt()
                    if self.tokens[0].code == 7:
                        self.tokens.pop(0)
                        self.stmt()
                else:
                    self.error()
            else:
                self.error()
        else:
            self.error()
    
    def do_while(self):
        # <do_while> --> `do` <block> <while_loop>
        if self.tokens[0].code == 10:
            self.tokens.pop(0)
            self.block()
            self.while_loop()
        else:
            self.error()
    
    def while_loop(self):
        # <while_loop> -->  `while``(`<bool_stmt>`)`<stmt>
        if self.tokens[0].code == 8:
            self.tokens.pop(0)
            if self.tokens[0].code == 33:
                self.tokens.pop(0)
                self.bool_stmt()
                if self.tokens[0].code == 34:
                    self.tokens.pop(0)
                    self.stmt()
                else:
                    self.error()
            else:
                self.error()
        else:
            self.error()
    
    def for_loop(self):
        # <for_loop> --> `for``(`<bool_stmt>`)`<block>
        if self.tokens[0].code == 9:
            self.tokens.pop(0)
            if self.tokens[0].code == 33:
                self.tokens.pop(0)
                self.bool_stmt()
                if self.tokens[0].code == 34:
                    self.tokens.pop(0)
                    self.block()
                else:
                    self.error()
            else:
                self.error()
        else:
            self.error()
    # <assignment> --> `var` <id> `=` <expr>
    def assignment(self):
        if self.tokens[0].code == 11:
            self.tokens.pop(0)
            if self.tokens[0].code == 39:
                self.tokens.pop(0)
                if self.tokens[0].code == 22:
                    self.tokens.pop(0)
                    self.expr()
                else:
                    self.error()
            else:
                self.error()
        else:
            self.error()
    
    def functions(self):
        # <functions> -- > `function` <id> `(` <id> `)` <block>
        if self.tokens[0].code == 38:
            self.tokens.pop(0)
            if self.tokens[0].code == 39:
                self.tokens.pop(0)
                if self.tokens[0].code == 33:
                    self.tokens.pop(0)
                    if self.tokens[0].code == 39:
                        self.tokens.pop(0)
                        if self.tokens[0].code == 34:
                            self.tokens.pop(0)
                            self.block()
                        else:
                            self.error()
                    else:
                        self.error()
                else:
                    self.error()
            else:
                self.error()
        else:
            self.error()
    # <expr> --> <term> {(`+`|`-`)<term>}
    def expr(self):
        if self.tokens[0].code == 39:
            self.term()
            while self.tokens[0].code == 21 or self.tokens[0].code == 20:
                self.tokens.pop(0)
                self.term()
        else:
            self.error()
    # <term> --> <val>{(`*`|`/`|`%`|`\*\*`)<val>}
    def term(self):
        if self.tokens[0].code == 39 or self.tokens[0].code == 40 or self.tokens[0].code == 41 or self.tokens[0].code == 42:
            self.val()
            while self.tokens[0].code == 23 or self.tokens[0].code == 24 or self.tokens[0].code == 25 or self.tokens[0].code == 26:
                self.tokens.pop(0)
                self.val()
        else:
            self.error()
    # <val> --> <id> | <real_literal> | <integer_literal> | <bool_literal> | <char_literal> | <string_literal> | `(` <expr> `)`
    def val(self):
        if self.tokens[0].code == 39:
            self.tokens.pop(0)
        elif self.tokens[0].code == 40:
            self.tokens.pop(0)
        elif self.tokens[0].code == 41:
            self.tokens.pop(0)
        elif self.tokens[0].code == 42:
            self.tokens.pop(0)
        elif self.tokens[0].code == 43:
            self.tokens.pop(0)
        elif self.tokens[0].code == 44:
            self.tokens.pop(0)
        elif self.tokens[0].code == 33:
            self.tokens.pop(0)
            self.expr()
            if self.tokens[0].code == 34:
                self.tokens.pop(0)
            else:
                self.error()
        else:
            self.error()
    # <bool_stmt> --> `True` | `False` | <expr> (`==`|`!==`|`<`|`>`|`<==`|`>==`) <expr> 
    def bool_stmt(self):
        if self.tokens[0].code == 42:
            self.tokens.pop(0)
        elif self.tokens[0].code == 41:
            self.tokens.pop(0)
        else:
            self.expr()
            if self.tokens[0].code == 27 or self.tokens[0].code == 28 or self.tokens[0].code == 29 or self.tokens[0].code == 30 or self.tokens[0].code == 31 or self.tokens[0].code == 32:
                self.tokens.pop(0)
                self.expr()
            else:
                self.error()
       

In [29]:
class Semantics:
    def __init__(self, tokens):
        self.tokens = tokens
        self.current_token = None
        self.next_token = None
        self.current_token_index = 0
        self.next_token_index = 1
        self.check_semantics()

        while(self.current_token_index < len(self.tokens)):  
            self.current_token = self.tokens[self.current_token_index]
            self.next_token = self.tokens[self.next_token_index]
            self.current_token_index += 1
            self.next_token_index += 1
            self.check_semantics()
    
    def check_semantics(self):
        if self.current_token.code == 39:
            if self.next_token.code == 22:
                self.next_token = self.tokens[self.next_token_index + 1]
                if self.next_token.code == 39:
                    print("Variable " + self.current_token.lexeme + " has been assigned the value " + self.next_token.lexeme)
                elif self.next_token.code == 40:
                    print("Variable " + self.current_token.lexeme + " has been assigned the value " + self.next_token.lexeme)
                elif self.next_token.code == 41:
                    print("Variable " + self.current_token.lexeme + " has been assigned the value " + self.next_token.lexeme)
                elif self.next_token.code == 42:
                    print("Variable " + self.current_token.lexeme + " has been assigned the value " + self.next_token.lexeme)
                elif self.next_token.code == 43:
                    print("Variable " + self.current_token.lexeme + " has been assigned the value " + self.next_token.lexeme)
                elif self.next_token.code == 44:
                    print("Variable " + self.current_token.lexeme + " has been assigned the value " + self.next_token.lexeme)
                elif self.next_token.code == 33:
                    print("Variable " + self.current_token.lexeme + " has been assigned the value " + self.next_token.lexeme)
                else:
                    print("Variable " + self.current_token.lexeme + " has been assigned the value " + self.next_token.lexeme)
            else:
                print("Variable " + self.current_token.lexeme + " has been declared")
        else:
            pass
    

In [30]:
class Compiler:
    def __init__(self, file:str):
        self.file = file
        text_file = open(file, "r")

        input = text_file.read()
        lex = Lexer(input)
        tokens = lex.find_tokens()
        print(tokens)
        parser = Parser(tokens)
        parser.program()
        semantics = Semantics(tokens)
        return semantics
        text_file.close
        

In [31]:
c = Compiler("success_tokens.txt")
print(c)

[<__main__.Token object at 0x00000202172BE970>, <__main__.Token object at 0x0000020216A90D00>, <__main__.Token object at 0x00000202172BE9D0>, <__main__.Token object at 0x00000202172BEF10>, <__main__.Token object at 0x00000202172BE9D0>, <__main__.Token object at 0x0000020216A90D00>, <__main__.Token object at 0x00000202172BE9D0>, <__main__.Token object at 0x00000202172BEF10>, <__main__.Token object at 0x00000202172BE9D0>, <__main__.Token object at 0x0000020216A90D00>, <__main__.Token object at 0x00000202172BE9D0>, <__main__.Token object at 0x00000202172BEF10>, <__main__.Token object at 0x00000202172BE9D0>, <__main__.Token object at 0x0000020216A90580>, <__main__.Token object at 0x00000202172BE9D0>, <__main__.Token object at 0x00000202172BEE50>]


ValueError: Syntax Error